In [65]:
# reference https://www.kaggle.com/qinhui1999/deep-learning-is-all-you-need-lb-0-80x


import numpy as np
import pandas as pd

import os
dir = '/Users/xinwang/ai/dataset/kaggle/DonorsChoose/'
print(os.listdir(dir))

os.environ['PYTHONHASHSEED'] = '10000'
np.random.seed(10001)

import random
import tensorflow as tf
random.seed(10002)

session_conf = tf.ConfigProto(intra_op_parallelism_threads=6, inter_op_parallelism_threads=5)
from keras import backend

tf.set_random_seed(10003)
backend.set_session(tf.Session(graph=tf.get_default_graph(), config=session_conf))

train = pd.read_csv(dir+'train.csv')
test = pd.read_csv(dir+'test.csv')
resources = pd.read_csv(dir+'resources.csv')

train = train.sort_values(by='project_submitted_datetime')

teachers_train = list(set(train.teacher_id.values))
teachers_test = list(set(test.teacher_id.values))
inter = set(teachers_train).intersection(teachers_test)


print('teachers_train size:%s, teachers_test size:%s, inter size:%s' % (len(teachers_train),
                                                                        len(teachers_test),
                                                                        len(inter)))

['resources.csv', 'sample_submission.csv', 'test.csv', 'train.csv', 'train_1.csv', 'train_2.csv']


/Users/xinwang/ai/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (11,12) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


teachers_train size:104414, teachers_test size:55508, inter size:27789


In [66]:
resources['total_price'] = resources['quantity'] * resources['price']

def process_resources_features(df, col):
    funcs = ['mean','count','sum','median']
    for f in funcs:
        df_temp = pd.DataFrame(resources[['id',col]].groupby('id').agg(f))
        df_temp['id'] = df_temp.index
        df_temp.rename(columns={col:col + '_' + f}, inplace=True)
        
        df = pd.merge(df, df_temp, on='id')
    return df

for col in ['total_price','quantity']:
    train = process_resources_features(train, col)
    test = process_resources_features(test, col)


train.head()

/Users/xinwang/ai/lib/python3.6/site-packages/ipykernel_launcher.py:14: FutureWarning: 'id' is both an index level and a column label.
Defaulting to column, but this will raise an ambiguity error in a future version
  
/Users/xinwang/ai/lib/python3.6/site-packages/ipykernel_launcher.py:15: FutureWarning: 'id' is both an index level and a column label.
Defaulting to column, but this will raise an ambiguity error in a future version
  from ipykernel import kernelapp as app


,id,teacher_id,teacher_prefix,school_state,project_submitted_datetime,project_grade_category,project_subject_categories,project_subject_subcategories,project_title,project_essay_1,...,teacher_number_of_previously_posted_projects,project_is_approved,total_price_mean,total_price_count,total_price_sum,total_price_median,quantity_mean,quantity_count,quantity_sum,quantity_median
0,p146022,f91a89d2b72a0f5ee14cd98bd8741fc2,Ms.,CA,2016-04-27 00:03:38,Grades 6-8,Math & Science,Applied Sciences,Robotics and Programming,I love giving my students experiences. A new e...,...,30,1,596.000000,1,596.00,596.00,4.000000,1,4,4.0
1,p244738,64ef8a335f7206366c52c39f5bfd09b7,Ms.,TX,2016-04-27 00:04:09,Grades PreK-2,"Literacy & Language, Math & Science","Foreign Languages, Mathematics",Help Us Finish Our 1st Year In School!,"We are getting closer to the end of the year, ...",...,1,1,28.786000,10,287.86,29.97,1.000000,10,10,1.0
2,p070708,1f802ff6be7973fb3624332e10549ed4,Mrs.,CA,2016-04-27 00:15:39,Grades PreK-2,Math & Science,Mathematics,Common Core Math Tools for Young Minds,"Our classroom is an engaging, vibrant, and div...",...,8,1,75.257143,7,526.80,49.99,2.857143,7,20,2.0
3,p083729,f72fac3c6011b17ad5b2037dfd18cd0b,Teacher,CA,2016-04-27 00:20:33,Grades PreK-2,"Literacy & Language, Math & Science","Literacy, Mathematics",Bring 21st Century Resources Into Kindergarten!,My students are young children eager to engage...,...,0,1,256.400000,2,512.80,256.40,2.000000,2,4,2.0
4,p205479,2bf07ba08945e5d8b2a3f269b2b3cfe5,Mrs.,CA,2016-04-27 00:27:36,Grades PreK-2,Math & Science,"Applied Sciences, Health & Life Science",Engineering STEAM into the Primary Classroom,I have been fortunate enough to use the Fairy ...,...,53,1,181.262500,4,725.05,144.00,1.000000,4,4,1.0


In [67]:
def process_datetime_features(df):
    df['project_submitted_datetime'] = pd.to_datetime(df['project_submitted_datetime'])
    df['year'] = df['project_submitted_datetime'].dt.year
    df['month'] =df['project_submitted_datetime'].dt.month
    df['day'] =df['project_submitted_datetime'].dt.day
    
    return df
    
train = process_datetime_features(train)
test = process_datetime_features(test)
    
train.head()

,id,teacher_id,teacher_prefix,school_state,project_submitted_datetime,project_grade_category,project_subject_categories,project_subject_subcategories,project_title,project_essay_1,...,total_price_count,total_price_sum,total_price_median,quantity_mean,quantity_count,quantity_sum,quantity_median,year,month,day
0,p146022,f91a89d2b72a0f5ee14cd98bd8741fc2,Ms.,CA,2016-04-27 00:03:38,Grades 6-8,Math & Science,Applied Sciences,Robotics and Programming,I love giving my students experiences. A new e...,...,1,596.00,596.00,4.000000,1,4,4.0,2016,4,27
1,p244738,64ef8a335f7206366c52c39f5bfd09b7,Ms.,TX,2016-04-27 00:04:09,Grades PreK-2,"Literacy & Language, Math & Science","Foreign Languages, Mathematics",Help Us Finish Our 1st Year In School!,"We are getting closer to the end of the year, ...",...,10,287.86,29.97,1.000000,10,10,1.0,2016,4,27
2,p070708,1f802ff6be7973fb3624332e10549ed4,Mrs.,CA,2016-04-27 00:15:39,Grades PreK-2,Math & Science,Mathematics,Common Core Math Tools for Young Minds,"Our classroom is an engaging, vibrant, and div...",...,7,526.80,49.99,2.857143,7,20,2.0,2016,4,27
3,p083729,f72fac3c6011b17ad5b2037dfd18cd0b,Teacher,CA,2016-04-27 00:20:33,Grades PreK-2,"Literacy & Language, Math & Science","Literacy, Mathematics",Bring 21st Century Resources Into Kindergarten!,My students are young children eager to engage...,...,2,512.80,256.40,2.000000,2,4,2.0,2016,4,27
4,p205479,2bf07ba08945e5d8b2a3f269b2b3cfe5,Mrs.,CA,2016-04-27 00:27:36,Grades PreK-2,Math & Science,"Applied Sciences, Health & Life Science",Engineering STEAM into the Primary Classroom,I have been fortunate enough to use the Fairy ...,...,4,725.05,144.00,1.000000,4,4,1.0,2016,4,27


In [68]:
char_columns = ['project_subject_categories','project_subject_subcategories','project_title','project_essay_1',
                'project_essay_2','project_essay_3','project_essay_4','project_resource_summary']

def process_text_features(df):
    for col in char_columns:
        df[col] = df[col].fillna('NA')
    
    df['text'] = df.apply(lambda x: " ".join(x[col] for col in char_columns), axis=1)
    
    return df
    
train = process_text_features(train)
test = process_text_features(test)
    
train.head()
    

,id,teacher_id,teacher_prefix,school_state,project_submitted_datetime,project_grade_category,project_subject_categories,project_subject_subcategories,project_title,project_essay_1,...,total_price_sum,total_price_median,quantity_mean,quantity_count,quantity_sum,quantity_median,year,month,day,text
0,p146022,f91a89d2b72a0f5ee14cd98bd8741fc2,Ms.,CA,2016-04-27 00:03:38,Grades 6-8,Math & Science,Applied Sciences,Robotics and Programming,I love giving my students experiences. A new e...,...,596.00,596.00,4.000000,1,4,4.0,2016,4,27,Math & Science Applied Sciences Robotics and P...
1,p244738,64ef8a335f7206366c52c39f5bfd09b7,Ms.,TX,2016-04-27 00:04:09,Grades PreK-2,"Literacy & Language, Math & Science","Foreign Languages, Mathematics",Help Us Finish Our 1st Year In School!,"We are getting closer to the end of the year, ...",...,287.86,29.97,1.000000,10,10,1.0,2016,4,27,"Literacy & Language, Math & Science Foreign La..."
2,p070708,1f802ff6be7973fb3624332e10549ed4,Mrs.,CA,2016-04-27 00:15:39,Grades PreK-2,Math & Science,Mathematics,Common Core Math Tools for Young Minds,"Our classroom is an engaging, vibrant, and div...",...,526.80,49.99,2.857143,7,20,2.0,2016,4,27,Math & Science Mathematics Common Core Math To...
3,p083729,f72fac3c6011b17ad5b2037dfd18cd0b,Teacher,CA,2016-04-27 00:20:33,Grades PreK-2,"Literacy & Language, Math & Science","Literacy, Mathematics",Bring 21st Century Resources Into Kindergarten!,My students are young children eager to engage...,...,512.80,256.40,2.000000,2,4,2.0,2016,4,27,"Literacy & Language, Math & Science Literacy, ..."
4,p205479,2bf07ba08945e5d8b2a3f269b2b3cfe5,Mrs.,CA,2016-04-27 00:27:36,Grades PreK-2,Math & Science,"Applied Sciences, Health & Life Science",Engineering STEAM into the Primary Classroom,I have been fortunate enough to use the Fairy ...,...,725.05,144.00,1.000000,4,4,1.0,2016,4,27,"Math & Science Applied Sciences, Health & Life..."


In [69]:
cate_features = ['teacher_prefix', 'school_state','year','month', 'day','project_grade_category',
                 'project_subject_categories', 'project_subject_subcategories']
cate_features_hash = [col+'_hash' for col in cate_features]

max_size=15000

def feature_hash(df, max_size=max_size):
    for col in cate_features:
        df[col+'_hash'] = df[col].apply(lambda x: hash(x)%max_size)
    return df

train = feature_hash(train)
test = feature_hash(test)

print(train.columns.values)

['id' 'teacher_id' 'teacher_prefix' 'school_state'
 'project_submitted_datetime' 'project_grade_category'
 'project_subject_categories' 'project_subject_subcategories'
 'project_title' 'project_essay_1' 'project_essay_2' 'project_essay_3'
 'project_essay_4' 'project_resource_summary'
 'teacher_number_of_previously_posted_projects' 'project_is_approved'
 'total_price_mean' 'total_price_count' 'total_price_sum'
 'total_price_median' 'quantity_mean' 'quantity_count' 'quantity_sum'
 'quantity_median' 'year' 'month' 'day' 'text' 'teacher_prefix_hash'
 'school_state_hash' 'year_hash' 'month_hash' 'day_hash'
 'project_grade_category_hash' 'project_subject_categories_hash'
 'project_subject_subcategories_hash']


In [70]:
import re
from sklearn.preprocessing import StandardScaler
from keras.preprocessing import text, sequence

max_features = 100000
maxlen = 300

def replace_text(string):
    string = re.sub(r'(\")', ' ', string)
    string = re.sub(r'(\r)', ' ', string)
    string = re.sub(r'(\n)', ' ', string)
    string = re.sub(r'(\r\n)', ' ', string)
    string = re.sub(r'(\\)', ' ', string)
    string = re.sub(r'\t', ' ', string)
    
    string = re.sub(r'\"\"\"\"', ' ', string)
    string = re.sub(r'_', ' ', string)
    string = re.sub(r'\+', ' ', string)
    string = re.sub(r'\=', ' ', string)
    
    return string


train['text'] = train['text'].apply(replace_text)
test['text'] = test['text'].apply(replace_text)
    
tokenizer = text.Tokenizer(num_words =max_features)
tokenizer.fit_on_texts(train['text'].tolist() + test['text'].tolist())

tokenized_text_train = tokenizer.texts_to_sequences(train['text'].tolist())
tokenized_text_test = tokenizer.texts_to_sequences(test['text'].tolist())

x_train_words = sequence.pad_sequences(tokenized_text_train, maxlen=maxlen)
x_test_words = sequence.pad_sequences(tokenized_text_test, maxlen=maxlen)

x_train_target = train.project_is_approved

In [71]:
word_index = tokenizer.word_index
print('word_index')

word_index


In [72]:
pretrain_word_vector_file= '/Users/xinwang/ai/dataset/kaggle/pretrain_text/crawl-300d-2M.vec'
embedding_size=300


def get_embedding_matrix():
    embedding_index = {}

    with open(pretrain_word_vector_file, encoding='UTF8') as f:
        for line in f:
            values = line.rstrip().rsplit(' ')
            word = values[0]

            coefs = np.asarray(values[1:], dtype='float32')

            embedding_index[word] = coefs

    word_index = tokenizer.word_index

    num_words = min(max_features, len(word_index)+1)
    embedding_matrix = np.zeros((num_words, embedding_size))

    for word, i in word_index.items():
        if i >= max_features:
            continue

        embedding_vector = embedding_index.get(word)

        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector

    return embedding_matrix
        

embedding_matrix = get_embedding_matrix()
print('done get embedding_matrix')

done get embedding_matrix


In [73]:
numeric_features = ['teacher_number_of_previously_posted_projects','total_price_mean', 
                    'total_price_count','total_price_sum', 'total_price_median', 'quantity_mean',
                    'quantity_count', 'quantity_sum', 'quantity_median']

scaler = StandardScaler()
x_train_numeric = scaler.fit_transform(train[numeric_features])
x_test_numeric = scaler.fit_transform(test[numeric_features])

x_train_cate = np.array(train[cate_features_hash], dtype=np.int)
x_test_cate = np.array(test[cate_features_hash], dtype=np.int)

In [74]:
from keras.layers import Input, Dense, Embedding, Flatten, concatenate, Dropout
from keras.layers import Convolution1D, GlobalMaxPool1D, SpatialDropout1D, CuDNNGRU, Bidirectional, PReLU, GRU
from keras.models import Model
from keras import optimizers


def get_model():
    input_cate = Input((len(cate_features_hash),))
    input_numeric = Input((len(numeric_features),))
    input_words = Input((maxlen,))
    
    x_cate = Embedding(max_size, 10)(input_cate)
    x_cate = SpatialDropout1D(0.2)(x_cate)
    x_cate = Flatten()(x_cate)
    x_cate = Dense(100, activation='relu')(x_cate)
    
    x_words = Embedding(max_features, 300, weights = [embedding_matrix],
                       trainable=False)(input_words)
    x_words = SpatialDropout1D(0.2)(x_words)
    x_words = Bidirectional(GRU(50, return_sequences=True))(x_words)
    x_words = Convolution1D(100, 3, activation='relu')(x_words)
    x_words = GlobalMaxPool1D()(x_words)
    
    x_numeric = Dense(100, activation='relu')(input_numeric)
    
    x = concatenate([x_cate, x_numeric, x_words])
    
    x = Dense(50, activation='relu')(x)
    x = Dropout(0.2)(x)
    
    predictions = Dense(1, activation='sigmoid')(x)
    
    model = Model(inputs=[input_cate,input_numeric,input_words], outputs=predictions)
    model.compile(optimizer=optimizers.Adam(0.0005, decay=1e-6),
                 loss='binary_crossentropy',
                 metrics=['accuracy'])
    
    return model

model = get_model()

print(model.summary())

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_44 (InputLayer)           (None, 300)          0                                            
__________________________________________________________________________________________________
input_42 (InputLayer)           (None, 8)            0                                            
__________________________________________________________________________________________________
embedding_28 (Embedding)        (None, 300, 300)     30000000    input_44[0][0]                   
__________________________________________________________________________________________________
embedding_27 (Embedding)        (None, 8, 10)        150000      input_42[0][0]                   
__________________________________________________________________________________________________
spatial_dr

In [75]:
from keras.callbacks import *
from sklearn.metrics import roc_auc_score

file_path='simpleRNN3.h5'
checkpoint = ModelCheckpoint(file_path,monitor='val_loss',verbose=2, save_best_only=True,
                            save_weights_only=True,mode='min')
early = EarlyStopping(monitor='val_loss', mode='min', patience=4)

lr_reduced = ReduceLROnPlateau(monitor='val_loss',
                              factor=0.1,
                              patience=2,
                              verbose=1,
                              epsillon=1e-4,
                              mode='min')
callbacks_list = [checkpoint,early,lr_reduced]

history = model.fit([x_train_cate,x_train_numeric,x_train_words],x_train_target,validation_split=0.1,
                   verbose=2,callbacks=callbacks_list, epochs=5, batch_size=256)

del x_train_numeric,x_train_cate,x_train_words

model.load_weights(file_path)

pre_test = model.predict([x_test_cate,x_test_numeric,x_test_words], batch_size=2000)
    
test['project_is_approved'] = pre_test
test[['id','project_is_approved']].to_csv('gru_cnn_submission.csv', index=False)

Train on 163872 samples, validate on 18208 samples
Epoch 1/5
 - 935s - loss: 0.3857 - acc: 0.8500 - val_loss: 0.3535 - val_acc: 0.8599

Epoch 00001: val_loss improved from inf to 0.35353, saving model to simpleRNN3.h5
Epoch 2/5
 - 905s - loss: 0.3578 - acc: 0.8585 - val_loss: 0.3388 - val_acc: 0.8633

Epoch 00002: val_loss improved from 0.35353 to 0.33884, saving model to simpleRNN3.h5
Epoch 3/5
 - 909s - loss: 0.3486 - acc: 0.8618 - val_loss: 0.3345 - val_acc: 0.8636

Epoch 00003: val_loss improved from 0.33884 to 0.33447, saving model to simpleRNN3.h5
Epoch 4/5
 - 913s - loss: 0.3427 - acc: 0.8635 - val_loss: 0.3334 - val_acc: 0.8640

Epoch 00004: val_loss improved from 0.33447 to 0.33344, saving model to simpleRNN3.h5
Epoch 5/5
 - 921s - loss: 0.3388 - acc: 0.8646 - val_loss: 0.3340 - val_acc: 0.8647

Epoch 00005: val_loss did not improve from 0.33344
